In [56]:
# Classes

class Card():
    '''A class to manage a single card'''
    
    def __init__(self,
                 value:str = None,
                 suit:str = None,
                 number:int = None):
        
        self.value = value
        self.suit = suit
        self.number = number
        self.name = self.value + ' of ' + self.suit
    
class Pile():
    '''A class to manage an ordered set of cards'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None):
        
        self.name = name
        
        if card_list is None:
            self.card_list = []
        else:
            self.card_list = card_list
        
    def shuffle(self):
        # This method is called to shuffle the pile
        
        random.shuffle(self.card_list)
        
class Hand(Pile):
    '''A class to manage a hand of cards, child class to Pile'''
    
    def __init__(self,
                 name:str = None,
                 card_list:list = None,
                 owner:str = None):
        
        Pile.__init__(self,name,card_list)
        self.name = owner + "'s Hand"
        self.owner = owner
        
    def draw(self,
            pile:str = None,
            number:int = 1):
        # This method is called to draw a number of cards to the hand from the draw pile
        
        self.card_list.extend(pile.card_list[-number:])
        del pile.card_list[-number:]
        
    def play_book(self,
                  pile:str = None,
                  played_cards:list = None):
        # This method is called to play a book of cards from the hand to the discard pile
        
        for card in played_cards:
            self.card_list.remove(card)
            pile.card_list.append(card)
            
    def sort(self,
             suit_sort:bool = False):
        # This method will sort the cards in a hand ascending, by default value then suit
        
        if suit_sort is True:
            self.card_list.sort(key=lambda x: (x.suit, x.number))
        else:
            self.card_list.sort(key=lambda x: (x.number, x.suit))
            
    def ask(self,
            value:str = None,
            target_hand:str = None):
        # This method is used to ask for a card from another player, "Do you have any X?"
        
        print('\n' + self.owner + ' asks ' + target_hand.owner + ' if they have any ' + value + '.')
        
        card_count = 0
        new_target_list = []
        for index, card in enumerate(target_hand.card_list.copy()):
            if card.value == value:
                self.card_list.append(card)
                card_count += 1
            else:
                new_target_list.append(card)
        target_hand.card_list = new_target_list
        
        if card_count > 0:
            print('\n' + target_hand.owner + ' gives ' + str(card_count) + ' ' + value + ' to ' + self.owner + '.' )
        else:
            print('\n' + target_hand.owner + ' has no ' + value + '. ' + self.owner + ', go fish!')
        
    
class Scoreboard():
    '''A class to manage the overall game state'''
    
    def __init__(self,
                players:list = None,
                book_values:dict = None):
        
        if players is None:
            self.players = []
        else:
            self.players = players
        
        self.score = {}
        for player in self.players:
            self.score[player] = 0
        
        self.books = {}
        for key, value in book_values.items():
            self.books[value] = 0
            
    def update_score(self,
                    player:str = None,
                    value:str = None):
        # This method is called to update the score and denote a played book with value 1
        
        self.score[player] += 1
        self.books[value] = 1
    
    def print_score(self):
        # This method is called to print a summary of the game state
        
        print('Score  Player')
        for player in sorted(self.score, key = self.score.get, reverse = True):
            print(str(self.score[player]) + '      ' + player)
            
        played = [key for key, value in self.books.items() if value == 1]
        unplayed = [key for key, value in self.books.items() if value == 0]
        
        if len(played) > 0 and len(played) < 13:
            print('\n')
            print('Books in play:')
            for book in played:
                print(book)
        if len(unplayed) > 0:
            print('\n')
            print('Available books:')
            for book in unplayed:
                print('  ' + book)

In [57]:
# Game initialization  
import random

def startgame(player_list = ['player1','player2']):
    # Set variables and check for valid player count
    cards, hands = [], []
    player_count = len(player_list)
        
    if player_count > 1 and player_count < 4:
        starting_hand_size = 7
    elif player_count > 3 and player_count < 7:
        starting_hand_size = 5
    else:
        return print('There must be between two and six players.')
    
    suits = {0:"Clubs", 1:"Diamonds", 2:"Hearts", 3:"Spades"}
    values = {0:"Two", 1:"Three", 2:"Four", 3:"Five", 4:"Six", 5:"Seven", 6:"Eight",
              7:"Nine", 8:"Ten", 9:"Jack", 10:"Queen", 11:"King", 12:"Ace"}
    
    # Create the 52 playing cards
    for s in range(4):
        for v in range(13):
            cards.append(Card(value = values[v], suit = suits[s], number = v))
    
    # Create piles, scoreboards, and hands
    draw_pile = Pile(name = "Draw Pile", card_list = cards.copy())
    discard_pile = Pile(name = "Discard Pile")
    scoreboard = Scoreboard(players = player_list, book_values = values)
    for player in scoreboard.players:
        hands.append(Hand(owner = player))
    
    # Shuffle up and deal and sort hands
    draw_pile.shuffle()
    for hand in hands:
        hand.draw(pile = draw_pile, number = starting_hand_size)
        hand.sort()
    
    return cards, draw_pile, discard_pile, scoreboard, hands

In [58]:
#Test area
cards, draw_pile, discard_pile, scoreboard, hands = startgame(['Alex','Louis','Erin'])

print('The Draw Pile has ' + str(len(draw_pile.card_list)) + ' cards and the top card is:')
print('  ' + draw_pile.card_list[-1].value + ' of ' + draw_pile.card_list[-1].suit)
print('\n')
for player in hands:
    print(player.name)
    for card in player.card_list:
        print('  ' + card.name)
    print('\n')

hands[0].play_book(pile = discard_pile, played_cards = hands[0].card_list[-4:])
print('After playing a "book", Alex has ' + str(len(hands[0].card_list)) + ' cards.')
print('Discard Pile:')
for card in discard_pile.card_list:
    print('  ' + card.name)
print('\n')
scoreboard.print_score()
hands[0].ask(value = 'Ace', target_hand = hands[1])

for player in hands:
    print(player.name)
    for card in player.card_list:
        print('  ' + card.name)
    print('\n')

The Draw Pile has 31 cards and the top card is:
  Ten of Diamonds


Alex's Hand
  Three of Diamonds
  Three of Hearts
  Seven of Clubs
  Eight of Diamonds
  Eight of Spades
  King of Clubs
  Ace of Clubs


Louis's Hand
  Two of Spades
  Three of Clubs
  Four of Clubs
  Nine of Spades
  Ten of Clubs
  Ace of Hearts
  Ace of Spades


Erin's Hand
  Three of Spades
  Six of Hearts
  Six of Spades
  Nine of Clubs
  Queen of Hearts
  King of Diamonds
  King of Spades


After playing a "book", Alex has 3 cards.
Discard Pile:
  Eight of Diamonds
  Eight of Spades
  King of Clubs
  Ace of Clubs


Score  Player
0      Alex
0      Louis
0      Erin


Available books:
  Two
  Three
  Four
  Five
  Six
  Seven
  Eight
  Nine
  Ten
  Jack
  Queen
  King
  Ace

Alex asks Louis if they have any Ace.

Louis gives 2 Ace to Alex.
Alex's Hand
  Three of Diamonds
  Three of Hearts
  Seven of Clubs
  Ace of Hearts
  Ace of Spades


Louis's Hand
  Two of Spades
  Three of Clubs
  Four of Clubs
  Nine of Spad

In [51]:
apples = ['alex','beta','can']
for number, card in enumerate(apples):
    print(number, card)

0 alex
1 beta
2 can
